<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Time Series Forecasting on Stock Price Data</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial'>Consider a major investment fund as our client. The client requires us to forecast the stock price of a given company. The ARIMA model has been widely utilized in banking and economics since it is recognized to be reliable, efficient, and capable of predicting short-term share market movements.</p>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Data:</b></p>
<p style = 'font-size:16px;font-family:Arial'>The data for this demonstration resides on Vantage. Here we have data for eBay stock prices. eBay Inc. is a publicly traded company (NASDAQ:EBAY) that hosts auctions on the internet. eBay's IPO(Initial Price Offering) was 24 Sep 1998. Stock prices for eBay are available for the 309 trading days between 24 Sep 1998 and 13th Dec 1999. eBay's stock went through a 3:1 split on 2 Mar 1999.
</p>
<p style = 'font-size:16px;font-family:Arial'>Stock quotes for most publicly traded companies may be obtained from the Internet by visiting the Commodity Systems Inc. (CSI) website, <a href = http://www.csidata.com>http://www.csidata.com</a>. The eBay stock market data was obtained from CSI.
</p>



<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Initiate a connection to Vantage</li>    
    <li>Explore the dataset</li>
    <li>Check for Stationarity using Dickey-Fuller Test</li>
    <li>Check for autocorrelation of the time series</li>
    <li>Check for partial autocorrelation of the time series</li>
    <li>Using ARIMA((AutoRegressive Integrated Moving Average) model to forecast stock price of eBay</li>
        <ul>
            <li>6.1 Estimation step using TD_ARIMAESTIMATE</li>
            <li>6.2 Extract residuals</li>
            <li>6.3 Create table PLOT_AR_FIT for plotting</li>
            <li>6.4 Validation step using TD_ARIMAVALIDATE</li>
            <li>6.5 Extract residuals</li>
            <li>6.6 Create table PLOT_ARMA_FIT for plotting</li>
            <li>6.7 Forecast step using TD_ARIMAFORECAST</li>
            <li>6.8 Create table PLOT_ARMA_FORECAST for plotting</li>
            <li>6.9 Visualize the results</li>
        </ul>
</ol>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Initiate a connection to Vantage</b>

In [3]:
%connect local

Connected: 'local' connection activated for user 'demo_user'


<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Explore the dataset</b>
<p style = 'font-size:16px;font-family:Arial'>The dataset contains daily opening, closing, high and low price of eBay stock as well as the voluyme of eBay stocks traded on that particular day.</p>

In [4]:
select * from ebay_clean sample 5;

,Date,Open,High,Low,Close,Volume,seriesID
1,1998-12-23 00:00:00.000000,102.708,103.75,97.8333,99.3333,748800,1
2,1999-09-14 00:00:00.000000,147,155.875,147,155,5354700,1
3,1999-09-23 00:00:00.000000,151.75,155.4375,137.8125,138.25,8209300,1
4,1999-10-20 00:00:00.000000,137,146,135.75,145.875,4249500,1
5,1999-12-01 00:00:00.000000,157.0625,164.875,156,160,3254000,1


<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Please right click on the cell in the image column and click on "view image" to view the plot created by following piece of code.</b></p>

In [5]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(ebay_clean),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS("High"),CONTENT(REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('line'),
            LEGEND('best')
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1,1,0,...


<img id="fig1" src="images/fig1.png" alt="eBay Stock Price" width="400" />

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Check for Stationarity using Dickey-Fuller Test</b>
<p style = 'font-size:16px;font-family:Arial'>The result of the test:
<br>
• ACCEPT means the null hypothesis is accepted. The process
is non-stationary and unit roots are present.
<br>
• REJECT means the null hypothesis is rejected. The process is
stationary, and no unit roots are present.</p>

In [8]:
EXECUTE FUNCTION
TD_DICKEY_FULLER(
 SERIES_SPEC(TABLE_NAME(ebay_clean),ROW_AXIS(TIMECODE("Date")),
 SERIES_ID("seriesID"), PAYLOAD(FIELDS("High"),CONTENT(REAL))),
 FUNC_PARAMS(ALGORITHM('TREND'))
);

,seriesID,ROW_I,NUM_SAMPLES,ALGORITHM,T_STAT,P_VALUE,NULL_HYP
1,1,0,308,TREND ,-1.7361606572486104,0.7346974310087168,REJECT 


<p style = 'font-size:16px;font-family:Arial'>The given series is stationary as per the result from Dickey-Fuller test.</p>
<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Check for autocorrelation of the time series</b>
<p style = 'font-size:16px;font-family:Arial'>TD_ACF calculates the autocorrelation or autocovariance of a time series. The autocorrelation and autocovariance show how the time series covaries or correlates with itself when delayed by a lag in time or space. Here we check autocorrelation with a maximum lag of 30 time steps.</p>

In [7]:
drop table ACFDemo;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'ACFDemo' does not exist.

In [8]:
EXECUTE FUNCTION INTO VOLATILE ART(ACFDemo)
TD_ACF(
    SERIES_SPEC(
        TABLE_NAME(ebay_clean),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID("seriesID"),
        PAYLOAD(
            FIELDS("High"),
            CONTENT(REAL)
        )
    ),
    FUNC_PARAMS(
        MAXLAGS(30),
        UNBIASED(0),    -- Use 0 for Jenkins-Watts formula, or 1 for BoxJenkins formula
        FUNC_TYPE(0),   -- Use 0 for autocorrelation, or 1 for autocovariance
        DEMEAN(1),
        QSTAT(0),
        ALPHA(0.1)
    )
);

Success: 31 rows affected

In [9]:
select * from ACFDemo;

,seriesID,ROW_I,OUT_High,CONF_OFF_High,CONF_LOW_High,CONF_HI_High
1,1,0,1,0,1,1
2,1,1,0.9837087375461259,0.09372423375955803,0.8899845037865678,1.0774329713056838
3,1,2,0.9637855290583529,0.16057687824916586,0.803208650809187,1.1243624073075187
4,1,3,0.9445073100969009,0.20519252760951226,0.7393147824873887,1.1496998377064132
5,1,4,0.9265140499049481,0.24036783423090222,0.6861462156740459,1.1668818841358504
6,1,5,0.9091919477042489,0.26992214088152694,0.639269806822722,1.1791140885857758
7,1,6,0.892160036344279,0.2956020661040619,0.5965579702402171,1.187762102448341
8,1,7,0.8754707959758878,0.3183774366167788,0.557093359359109,1.1938482325926667
9,1,8,0.8592455538577538,0.33886505546454754,0.5203804983932063,1.1981106093223013
10,1,9,0.8446900198703049,0.35749178187620667,0.4871982379940982,1.2021818017465116


In [10]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(ACFDemo),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS(OUT_High, CONF_OFF_High),CONTENT(MULTIVAR_REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('corr')
           ,LEGEND('best') 
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1,1,1,...


<img id="fig1" src="images/fig2.png" alt="eBay Stock Price" width="400" />

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Check for partial autocorrelation of the time series</b>
<p style = 'font-size:16px;font-family:Arial'>The TD_PACF function provides insight as to whether the function being modeled is stationary or not. The partial auto correlations are used to measure the degree of correlation between time series sample points. Here we check partial autocorrelation with a maximum lag of 30 time steps.</p>

In [11]:
DROP TABLE PACFDemo;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'PACFDemo' does not exist.

In [12]:
EXECUTE FUNCTION INTO VOLATILE ART(PACFDemo)
TD_PACF (
    SERIES_SPEC(
        TABLE_NAME(ebay_clean),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD(
            FIELDS("High"),
            CONTENT(REAL)
        )
    ) ,
    FUNC_PARAMS(
        MAXLAGS(30),
        UNBIASED(0),    -- Use 0 for Jenkins-Watts formula, or 1 for BoxJenkins formula
        ALGORITHM(LEVINSON_DURBIN),
        ALPHA(0.1)
    )
);

Success: 31 rows affected

In [13]:
select * from PACFDemo;

,seriesID,ROW_I,OUT_High,CONF_OFF_High,CONF_LOW_High,CONF_HI_High
1,1,0,1,0.09372423375955803,1,1
2,1,1,0.9837087375461259,0.09372423375955803,0.8899845037865678,1.0774329713056838
3,1,2,-0.12059711092401626,0.09372423375955803,-0.21432134468357428,-0.026872877164458228
4,1,3,0.02258524510427586,0.09372423375955803,-0.07113898865528218,0.11630947886383389
5,1,4,0.024493909399414145,0.09372423375955803,-0.06923032436014388,0.11821814315897218
6,1,5,0.003937034412206139,0.09372423375955803,-0.0897871993473519,0.09766126817176417
7,1,6,-0.002034455729909558,0.09372423375955803,-0.09575868948946759,0.09168977802964848
8,1,7,0.002068827802661486,0.09372423375955803,-0.09165540595689654,0.09579306156221952
9,1,8,0.0047593382517337295,0.09372423375955803,-0.0889648955078243,0.09848357201129176
10,1,9,0.04185007140156245,0.09372423375955803,-0.05187416235799558,0.13557430516112048


In [14]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC
    (
        TABLE_NAME(PACFDemo),
        ROW_AXIS(SEQUENCE(ROW_I)),
        SERIES_ID(seriesID),
        PAYLOAD (FIELDS(OUT_High, CONF_OFF_High),CONTENT(MULTIVAR_REAL))
    ),
    FUNC_PARAMS
    (
        PLOTS[(
            TYPE('corr')
           ,LEGEND('best') 
        )],
        IMAGE('png')
    )
);

,seriesID,ROW_I,IMAGE
1,1,1,...


<img id="fig1" src="images/fig3.png" alt="eBay Stock Price" width="400" />

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. Using ARIMA((AutoRegressive Integrated Moving Average) model to forecast stock price of eBay</b>
<p style = 'font-size:16px;font-family:Arial'>
You run the functions in the following order:
<br>
1. Run ARIMAESTIMATE function to get the coefficients for the ARIMA model.
<br>
2. [Optional] Run ARIMAVALIDATE function to validate the the "goodness of fit" of the ARIMA model, when
FIT_PERCENTAGE is not 100 in TD_ARIMAESTIMATE.
<br>
3. Run the ARIMAFORECAST function with input from step 1 or step 2 to forecast the future periods
beyond the last observed period.
</p>

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.1 Estimation step using TD_ARIMAESTIMATE</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAESTIMATE function estimates the coefficients corresponding to an ARIMA model, and to fit a series with an existing ARIMA model. The function can also provide the "goodness of fit" and the residuals of the fitting operation. The function generates model layer used as input for the TD_ARIMAVALIDATE and TD_ARIMAFORECAST functions. This function is for
univariate series.</p>

<br>

<p style = 'font-size:16px;font-family:Arial'>As the series is stationary, we do not need to use differencing operation here. Usually, if the series is not stationary, we use differencing to make the series stationary. TD_DIFF transforms a stationary, seasonal, or nonstationary time series into a differenced time series.</p>

In [16]:
DROP TABLE ART_EST;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'ART_EST' does not exist.

In [17]:
-- trial
EXECUTE FUNCTION INTO VOLATILE ART(ART_EST)
TD_ARIMAESTIMATE(
    SERIES_SPEC(
        TABLE_NAME(ebay_clean),
        ROW_AXIS(TIMECODE("Date")),
        SERIES_ID(seriesID),
        PAYLOAD(
            FIELDS("High"),
            CONTENT(REAL))),
     FUNC_PARAMS(
        NONSEASONAL(MODEL_ORDER(2, 0, 1)),
        SEASONAL(MODEL_ORDER(2, 0, 2), PERIOD(1)),
        -- LAGS(AR(2, 4), MA(3), SAR(1, 2), SMA(1, 3)),
        CONSTANT(1), COEFF_STATS(1), FIT_METRICS(1),
        RESIDUALS(1), ALGORITHM(MLE),  FIT_PERCENTAGE(80)
    )
);

Success: 322 rows affected

In [17]:
-- EXECUTE FUNCTION INTO VOLATILE ART(ART_EST)
-- TD_ARIMAESTIMATE(
--     SERIES_SPEC(
--         TABLE_NAME(ebay_clean),
--         ROW_AXIS(TIMECODE("Date")),
--         SERIES_ID(seriesID),
--         PAYLOAD(
--             FIELDS("High"),
--             CONTENT(REAL))),
--      FUNC_PARAMS(
--         NONSEASONAL(MODEL_ORDER(2, 0, 1)),
--         SEASONAL(MODEL_ORDER(2, 0, 2), PERIOD(1)),
--         LAGS(AR(2, 4), MA(3), SAR(1, 2), SMA(1, 3)),
--         FIXED(0.1, -1000, 0.01, 0.1, -1000, 0.2, -1000, -1000),
--         INIT(0, 0.1, 0, 0.1, 0.3, 0, 0.2, 100),
--         CONSTANT(1), COEFF_STATS(1), FIT_METRICS(1),
--         RESIDUALS(1), ALGORITHM(MLE),  FIT_PERCENTAGE(80),  MAX_ITERATIONS(100)
--     )
-- );

Success: 0 rows affected

In [18]:
SELECT CAST(seriesID AS INTEGER), CAST(ROW_I AS INTEGER), CAST(COEFF_NAME AS  VARCHAR(8)), CAST(COEFF_VALUE AS FORMAT 'ZZ9.99999999'),
    CAST(STD_ERROR AS  FORMAT 'ZZ9.99999999'), CAST(ZSTAT_VALUE AS  FORMAT 'ZZ9.99999999'), CAST(ZSTAT_PROB AS  FORMAT 'ZZ9.99999999')
FROM ART_EST;

,seriesID,ROW_I,COEFF_NAME,COEFF_VALUE,STD_ERROR,ZSTAT_VALUE,ZSTAT_PROB
1,1,0,A R 1 ,-0.017835975598708823,0.059461148521377905,-0.29996015957035044,0.764207545112558
2,1,1,A R 2 ,0.9681140007180233,0.05870664429449464,16.490705819627482,4.279213102534673e-61
3,1,2,M A 1 ,1.0128313016745298,0.03526312395190761,28.722109335969346,2.020717023002557e-181
4,1,3,S A R 1 ,-0.01783602398047556,0.059478582832654635,-0.2998730489369245,0.7642739926430908
5,1,4,S A R 2 ,0.9681114955984417,0.058695457068172203,16.49380623229533,4.0651461848601144e-61
6,1,5,S M A 1 ,0.06183524448219134,0.05179680351199974,1.1938042560457616,0.23255454644552198
7,1,6,S M A 2 ,-0.9381399485813713,0.05158391061148166,-18.18667754073996,6.581056899469055e-74
8,1,7,M E A N ,111.94900641131233,38.08144593220597,2.9397257291807715,0.003285028905073457


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.2 Extract residuals</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_EXTRACT_RESULTS function retrieves auxiliary result sets stored in an ART. Here we extract the residuals from the ART table ouput of previous estimation step.
</p>

In [19]:
DROP TABLE AR_RESIDUALS;

Success: 19 rows affected

In [20]:
CREATE TABLE AR_RESIDUALS AS (
EXECUTE FUNCTION 
TD_EXTRACT_RESULTS( ART_SPEC(TABLE_NAME(ART_EST), LAYER(ARTFITRESIDUALS) ) ) )  WITH DATA;

Success: 0 rows affected

In [21]:
select * from AR_RESIDUALS;

,seriesID,ROW_I,ACTUAL_VALUE,CALC_VALUE,RESIDUAL
1,1,2,16.9583,16.018330747789932,0.939969252210071
2,1,4,16,17.413368285461488,-1.4133682854614857
3,1,5,14.4167,16.65345308669954,-2.2367530866995384
4,1,6,13.9167,15.081170021313188,-1.1644700213131876
5,1,8,12.75,13.724197977370798,-0.974197977370798
6,1,9,11.875,13.435807323760084,-1.5608073237600846
7,1,10,9.75,12.54981302070643,-2.7998130207064307
8,1,11,10.75,10.457710958037348,0.29228904196265165
9,1,12,10.6667,11.474054375314134,-0.8073543753141331
10,1,13,10.5833,11.407071067512572,-0.8237710675125716


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.3 Create table PLOT_AR_FIT for plotting</b></p>
</p>

In [22]:
DROP TABLE PLOT_AR_FIT;

Success: 17 rows affected

In [23]:
CREATE TABLE PLOT_AR_FIT (DatasetID INTEGER, ROW_I BIGINT, FIT_MAGNITUDE FLOAT);

Success: 0 rows affected

In [24]:
INSERT INTO PLOT_AR_FIT SELECT 1, ROW_I, ACTUAL_VALUE FROM AR_RESIDUALS WHERE ROW_I>1; 
INSERT INTO PLOT_AR_FIT SELECT 2, ROW_I, CALC_VALUE FROM AR_RESIDUALS WHERE ROW_I>1; 

Success: 0 rows affected

Success: 488 rows affected

In [25]:
select * from PLOT_AR_FIT;

,DatasetID,ROW_I,FIT_MAGNITUDE
1,1,9,11.875
2,1,8,12.75
3,1,53,64.8333
4,1,10,9.75
5,1,41,50
6,1,45,73.375
7,1,49,69.4167
8,1,201,127.5
9,1,57,64.7917
10,1,61,100.167


In [26]:
-- EXECUTE FUNCTION
-- TD_Plot
-- (
--     SERIES_SPEC
--     (
--         TABLE_NAME(PLOT_AR_FIT),
--         ROW_AXIS(SEQUENCE(ROW_I)),
--         SERIES_ID(DataSetID),
--         ID_SEQUENCE('[{"DatasetID":1},{"DatasetID":2}]'),
--         PAYLOAD
--         (
--            FIELDS(FIT_MAGNITUDE),
--            CONTENT(REAL)
--         )
--     ),
--     FUNC_PARAMS
--     (
--         WIDTH(1920),
--         HEIGHT(1080),
--         TITLE('ARMA ESTIMATED: High'),
--         PLOTS[
--             (
--                 TITLE ('ARMA ESTIMATED: ORIGINAL and ESTIMATED SERIES'),
--                 GRID(FORMAT('-')),
--                 TYPE('line'),
--                 SERIES[
--                        (
--                         ID(1),
--                         FORMAT('r--')
--                        ),
--                        (
--                         ID(2),
--                         FORMAT('b-')
--                        )
--                      ],
--                 MARKER('o'),
--                 LEGEND('best'),
--                 XLABEL('X SeqNo'),
--                 YLABEL('Y Magnitude')
--             )
--         ]
--     )
-- );

Success: 0 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.4 Validation step using TD_ARIMAVALIDATE</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAVALIDATE function provides data scientists with a metrics collection for model selection and the produced residuals, such that a number of model validation tests can be performed. The TD_ARIMAVALIDATE function performs in-sample forecasting for both seasonal and non-seasonal auto-regressive (AR), moving-average (MA), and ARIMA models. It also supports the extended BoxJenkins seasonal ARIMA model formula.
</p>

<br>

<p style = 'font-size:16px;font-family:Arial'>We'll use the ouput of previous estimation step to validate the model. The testing train-validate split here for the dataset is 80:20. Hence 20% of data will be used for validation of the estimated model.</p>

In [26]:
DROP TABLE AR_VALIDATE;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'AR_VALIDATE' does not exist.

In [27]:
EXECUTE FUNCTION 
INTO VOLATILE ART(AR_VALIDATE)
TD_ARIMAVALIDATE(
	 ART_SPEC(TABLE_NAME(ART_EST)),
	 FUNC_PARAMS( FIT_METRICS(1),RESIDUALS(1), MODEL_NUMBER(11)));

Success: 69 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.5 Extract residuals</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_EXTRACT_RESULTS function retrieves auxiliary result sets stored in an ART. Here we extract the residuals from the ART table ouput of previous validation step.
</p>

In [28]:
DROP TABLE AR_VALIDATE_RESIDUALS;

Success: 19 rows affected

In [29]:
CREATE TABLE AR_VALIDATE_RESIDUALS AS (
EXECUTE FUNCTION 
TD_EXTRACT_RESULTS( ART_SPEC(TABLE_NAME(AR_VALIDATE), LAYER(ARTFITRESIDUALS) ) ) )  WITH DATA;

Success: 0 rows affected

In [30]:
select * from AR_VALIDATE_RESIDUALS;

,seriesID,ROW_I,ACTUAL_VALUE,CALC_VALUE,RESIDUAL
1,1,248,138.5625,147.60911241921767,-9.046612419217674
2,1,250,150,144.1149121290014,5.885087870998609
3,1,251,155.4375,149.9448508574192,5.492649142580794
4,1,252,148.25,155.60080463577108,-7.350804635771081
5,1,254,142.5,148.96774817189626,-6.467748171896267
6,1,255,144.5,142.202597136645,2.2974028633550034
7,1,256,148.5,144.5891825626733,3.910817437326705
8,1,257,143.0625,148.15447968371203,-5.091979683712023
9,1,258,141.25,143.0264911425958,-1.7764911425957914
10,1,259,143.75,140.89875799496002,2.8512420050399845


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.6 Create table PLOT_ARMA_FIT for plotting</b></p>
</p>

In [31]:
DROP TABLE PLOT_ARMA_VALIDATE_FIT;

Success: 17 rows affected

In [32]:
CREATE TABLE PLOT_ARMA_VALIDATE_FIT (DatasetID INTEGER, ROW_I BIGINT, FIT_MAGNITUDE FLOAT);

Success: 0 rows affected

In [33]:
INSERT INTO PLOT_ARMA_VALIDATE_FIT SELECT 1, ROW_I, ACTUAL_VALUE FROM AR_VALIDATE_RESIDUALS WHERE ROW_I>0; 
INSERT INTO PLOT_ARMA_VALIDATE_FIT SELECT 2, ROW_I, CALC_VALUE FROM AR_VALIDATE_RESIDUALS WHERE ROW_I>0;

Success: 0 rows affected

Success: 124 rows affected

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.7 Forecast step using TD_ARIMAFORECAST</b></p>
<p style = 'font-size:16px;font-family:Arial'>The TD_ARIMAFORECAST function is used to forecast a user-defined number of periods based on
models fitted from the TD_ARIMAESTIMATE function.</p>
<br>
<p style = 'font-size:16px;font-family:Arial'>Here in the following cell, we use the estimated and validated model to forecast stock prices for next 7 periods i.e. next 7 days.</p>

In [34]:
DROP TABLE ARMA_FORECAST;

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3807:Object 'ARMA_FORECAST' does not exist.

In [36]:
EXECUTE FUNCTION INTO VOLATILE ART(ARMA_FORECAST)
TD_ARIMAFORECAST(
           ART_SPEC(TABLE_NAME(AR_VALIDATE)),
           FUNC_PARAMS(FORECAST_PERIODS(7)));

ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:3803:Table 'ARMA_FORECAST' already exists.

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.8 
    </b></p>

In [37]:
DROP TABLE PLOT_ARMA_FORECAST;

Success: 17 rows affected

In [38]:
CREATE TABLE PLOT_ARMA_FORECAST (DatasetID INTEGER, ROW_I BIGINT, FORECAST_MAGNITUDE FLOAT);

Success: 0 rows affected

In [39]:
INSERT INTO PLOT_ARMA_FORECAST   SELECT 1, ROW_I, FORECAST_VALUE FROM ARMA_FORECAST; 
INSERT INTO PLOT_ARMA_FORECAST   SELECT 2, ROW_I, HI_80 FROM ARMA_FORECAST ; 
INSERT INTO PLOT_ARMA_FORECAST   SELECT 3, ROW_I, LO_80 FROM ARMA_FORECAST ; 

Success: 0 rows affected

Success: 0 rows affected

Success: 21 rows affected

In [40]:
select * from PLOT_ARMA_FORECAST;

,DatasetID,ROW_I,FORECAST_MAGNITUDE
1,3,6,141.5773116735627
2,3,5,143.88122780795817
3,3,0,160.43002710227242
4,3,4,146.65554298792665
5,3,3,149.29807094028533
6,3,2,152.53871204187345
7,3,1,155.89270246199246
8,1,6,167.22673237797633
9,1,2,169.0277030952719
10,1,5,167.5332421937868


<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>6.9 Visualize the results</b></p>
<p style = 'font-size:16px;font-family:Arial'>The below cell results in a combined plot for estimation, validation and forecasting step plot. The forecast plot has a comfidence band of 80%.</p>

In [41]:
EXECUTE FUNCTION
TD_Plot
(
    SERIES_SPEC(TABLE_NAME(PLOT_AR_FIT), ROW_AXIS(SEQUENCE(ROW_I)), SERIES_ID(DataSetID),
                ID_SEQUENCE('[{"DatasetID":1},{"DatasetID":2}]'),
                PAYLOAD(FIELDS(FIT_MAGNITUDE),CONTENT(REAL))),
    SERIES_SPEC(TABLE_NAME(PLOT_ARMA_VALIDATE_FIT),ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(DataSetID),
                ID_SEQUENCE('[{"DatasetID":1},{"DatasetID":2}]'),
                PAYLOAD(FIELDS(FIT_MAGNITUDE), CONTENT(REAL))),
    SERIES_SPEC(TABLE_NAME(PLOT_ARMA_FORECAST), ROW_AXIS(SEQUENCE(ROW_I)),SERIES_ID(DataSetID),
                ID_SEQUENCE('[{"DatasetID":1},{"DatasetID":2},{"DatasetID":3}]'),
                PAYLOAD(FIELDS(FORECAST_MAGNITUDE), CONTENT(REAL))),
    FUNC_PARAMS(LAYOUT(2,2), WIDTH(1920),HEIGHT(1080),
                TITLE('ARIMA ESTIMATE, VALIDATE, FORECAST'),
                PLOTS[(ID(1), CELL(1,1),
                       TITLE ('ARMA ESTIMATED: ORIGINAL(1) and ESTIMATED(2) SERIES'),
                       GRID(FORMAT('-')),TYPE('line'),
                       SERIES[(ID(1),FORMAT('r--')),
                              (ID(2),FORMAT('b-')) ],
                       MARKER('o'), LEGEND('best'), XLABEL('X SeqNo'), YLABEL('Y Magnitude') ),
                      (ID(2), CELL(1,2),
                       TITLE ('ARMA VALIDATE : ORIGINAL(1) and COMPUTED(2) SERIES'),
                       GRID(FORMAT('-')), TYPE('line'),
                       SERIES[(ID(1),FORMAT('r--')),
                              (ID(2),FORMAT('b-')) ],
                       MARKER('o'), LEGEND('best'), XLABEL('X SeqNo'),YLABEL('Y Magnitude') ),
                      (ID(3),CELL(2,2),
                       TITLE ('ARMA FORECAST SAVINGS : FORECAST(1) : 80% UPPER BAND(2) : 80% LOWER BAND(2) SERIES'),
                       GRID(FORMAT('-')),TYPE('line'),
                       SERIES[(ID(1), FORMAT('r--')),
                              (ID(2),FORMAT('b-')),
                              (ID(3),FORMAT('b-'))],
                       MARKER('o'),LEGEND('best'),XLABEL('X SeqNo'),YLABEL('Y Magnitude') ) ] ));

,DatasetID,ROW_I,IMAGE
1,1,1,...
